<a href="https://colab.research.google.com/github/Tongxi-Hu/deep-learning/blob/main/d2l/5_deep_learning_computation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5-1 layers and blocks

In [1]:
## multilayer percepton
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net=nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
X=torch.rand(2,20)
net(X)

tensor([[ 0.2232,  0.0317, -0.0988,  0.3621,  0.1692, -0.0719, -0.1882,  0.1908,
          0.1450,  0.0837],
        [ 0.2225, -0.0724, -0.0713,  0.1952,  0.0758,  0.0634, -0.1972, -0.0568,
          0.0739,  0.1815]], grad_fn=<AddmmBackward0>)

In [4]:
class MLP(nn.Module):
    # Declare a layer with model parameters. Here, we declare two fully
    # connected layers
    def __init__(self):
        # Call the constructor of the `MLP` parent class `Module` to perform
        # the necessary initialization. In this way, other function arguments
        # can also be specified during class instantiation, such as the model
        # parameters, `params` (to be described later)
        super().__init__()
        self.hidden=nn.Linear(20,256)
        self.out=nn.Linear(256,10)
    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input `X`
    def forward(self,X):
        # Note here we use the funtional version of ReLU defined in the
        # nn.functional module.
        return self.out(F.relu(self.hidden(X)))

In [5]:
net = MLP()
net(X)

tensor([[-0.0829, -0.2163, -0.1311,  0.1132,  0.0270,  0.0298, -0.2432, -0.1505,
         -0.0782, -0.0304],
        [ 0.0010, -0.1621, -0.0776,  0.1150,  0.0025,  0.1088, -0.2152, -0.1116,
         -0.0108, -0.0036]], grad_fn=<AddmmBackward0>)

In [6]:
from torch.nn.modules import module
class MySequential(nn.Module):
    def __init__(self,*args) -> None:
        super().__init__()
        for idx,module in enumerate(args):
            # Here, `module` is an instance of a `Module` subclass. We save it
            # in the member variable `_modules` of the `Module` class, and its
            # type is OrderedDict
            self._modules[str(idx)]=module
    def forward(self,X):
        # OrderedDict guarantees that members will be traversed in the order
        # they were added
        for block in self._modules.values():
            X=block(X)
        return X

In [7]:
net=MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,20))
net(X)

tensor([[ 0.1160, -0.1280, -0.0667, -0.0502,  0.2994,  0.2167, -0.1540,  0.0238,
         -0.4126, -0.0932, -0.1627,  0.0950,  0.1384,  0.0337, -0.0871,  0.0199,
          0.1592,  0.0065, -0.0370, -0.0125],
        [ 0.1223, -0.1108, -0.0942, -0.1103,  0.1371,  0.1964, -0.1395, -0.0021,
         -0.2675, -0.0115, -0.1392,  0.1187,  0.1318,  0.1142, -0.1410,  0.0604,
          0.1688, -0.0542, -0.0692, -0.0357]], grad_fn=<AddmmBackward0>)

In [10]:
class FixedHiddenMLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight=torch.rand((20,20),requires_grad=False)
        self.linear=nn.Linear(20,20)
    def forward(self,X):
        X=self.linear(X)
        # Use the created constant parameters, as well as the `relu` and `mm`
        # functions
        X=F.relu(torch.mm(X,self.rand_weight)+1)
        # Reuse the fully-connected layer. This is equivalent to sharing
        # parameters with two fully-connected layers
        X=self.linear(X)
        while X.abs().sum()>1:
            X/=2
        return X.sum()

net=FixedHiddenMLP()
net(X)

tensor(-0.0943, grad_fn=<SumBackward0>)

In [11]:
class NestMLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.net=nn.Sequential(nn.Linear(20,64),nn.ReLU(),nn.Linear(64,32),nn.ReLU())
        self.linear=nn.Linear(32,16)
    def forward(self,X):
        return self.linear(self.net(X))
chimera=nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(X)

tensor(0.0791, grad_fn=<SumBackward0>)

# 5-2 parameter management

In [12]:
import torch 
from torch import nn

net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X=torch.rand((2,4))
net(X)

tensor([[-0.2479],
        [-0.2379]], grad_fn=<AddmmBackward0>)

In [15]:
print(net[2].state_dict()) # get the parameter of the second linear layer

OrderedDict([('weight', tensor([[ 0.1515,  0.1838,  0.1308,  0.0517, -0.2851, -0.1518,  0.0451,  0.0468]])), ('bias', tensor([-0.3221]))])


In [17]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.3221], requires_grad=True)
tensor([-0.3221])


In [18]:
net[2].weight.grad==None

True

In [19]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [20]:
net.state_dict()['2.bias'].data

tensor([-0.3221])

In [21]:
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())
def block2():
    net=nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}',block1())
    return net
rgnet=nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)

tensor([[-0.0459],
        [-0.0460]], grad_fn=<AddmmBackward0>)

In [22]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [25]:
rgnet[0][1][2].bias.data

tensor([-0.2523, -0.3372,  0.2238, -0.1471])

In [28]:
def init_normal(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([-0.0041,  0.0156, -0.0063, -0.0037]), tensor(0.))

In [29]:
def init_constant(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [30]:
def xavier(m):
    if type(m)==nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,42)
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.6742,  0.3078,  0.6825, -0.6636])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [31]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000,  7.4937, -0.0000, -5.6572],
        [ 0.0000,  8.3426,  5.8051,  0.0000]], grad_fn=<SliceBackward0>)

In [32]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  8.4937,  1.0000, -4.6572])

In [34]:
# We need to give the shared layer a name so that we can refer to its parameters
shared=nn.Linear(8,8)
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),shared,nn.ReLU(),shared,nn.ReLU(),nn.Linear(8,1))
net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0]==net[4].weight.data[0])
net[2].weight.data[0,0]=100
# Make sure that they are actually the same object rather than just having the same value
print(net[2].weight.data[0]==net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


# 5-4 custom layers

In [38]:
import torch
from torch import nn
from torch.nn import functional as F

class CenteredLayer(nn.Module):
    def __init__(self) -> None:
        super().__init__()
    def forward(self,X):
        return X-X.mean()

In [39]:
layer=CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [41]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())
Y = net(torch.rand(4, 8))
Y.mean()

tensor(7.6834e-09, grad_fn=<MeanBackward0>)

In [42]:
class MyLinear(nn.Module):
    def __init__(self,in_units,units) -> None:
        super().__init__()
        self.weight=nn.Parameter(torch.randn(in_units,units))
        self.bias=nn.Parameter(torch.randn(units,))
    def forward(self,X):
        linear=torch.matmul(X,self.weight.data)+self.bias.data
        return F.relu(linear)

In [43]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.7518, -0.1617, -0.8676],
        [-1.5903, -0.4214,  0.2124],
        [ 1.3349,  0.0058,  0.2909],
        [ 0.4538,  0.8678, -0.0586],
        [-1.4376,  0.5825,  0.7355]], requires_grad=True)

In [44]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.0000, 0.0000],
        [0.0000, 0.2261, 0.0000]])

In [45]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.4288],
        [0.0000]])

# 5-5 file I/O

In [46]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [47]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [48]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [49]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [50]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [51]:
torch.save(net.state_dict(), 'mlp.params')

In [53]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [54]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

# 5-6 GPUs

In [1]:
!nvidia-smi

Sat Feb 12 01:39:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [3]:
torch.cuda.device_count()

1

In [4]:
def try_gpu(i=0): 
    """Return gpu(i) if exists, otherwise return cpu()."""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [12]:
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.1685, 0.2979, 0.8569],
        [0.9933, 0.6916, 0.4523]], device='cuda:0')

In [14]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [15]:
Y + Z

tensor([[1.1685, 1.2979, 1.8569],
        [1.9933, 1.6916, 1.4523]], device='cuda:0')

In [16]:
Z.cuda(0) is Z

True

In [17]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [18]:
net(X)

tensor([[-0.3647],
        [-0.3647]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [20]:
net[0].weight.data.device

device(type='cuda', index=0)